In [35]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OrdinalEncoder

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [2]:
inpatient_data = pd.read_pickle('../../data/feature engineered/inpatient.pkl')
outpatient_data = pd.read_pickle('../../data/feature engineered/outpatient.pkl')

****Preprocessing****

In [3]:
inpatient_data.dtypes

ClaimID                                    object
ClaimStartDt                       datetime64[ns]
ClaimEndDt                         datetime64[ns]
Provider                                   object
InscClaimAmtReimbursed                      int64
AttendingPhysician                        float64
OperatingPhysician                        float64
OtherPhysician                            float64
AdmissionDt                        datetime64[ns]
ClmAdmitDiagnosisCode                     float64
DeductibleAmtPaid                         float64
DischargeDt                        datetime64[ns]
DiagnosisGroupCode                        float64
ClmDiagnosisCode_1                        float64
ClmDiagnosisCode_2                        float64
ClmDiagnosisCode_3                        float64
ClmDiagnosisCode_4                        float64
ClmDiagnosisCode_5                        float64
ClmDiagnosisCode_6                        float64
ClmDiagnosisCode_7                        float64


In [3]:
in_standardizing = ['IPAnnualReimbursementAmt',
                    'IPAnnualDeductibleAmt',
                    'OPAnnualReimbursementAmt',
                    'OPAnnualDeductibleAmt']

In [4]:
std_scale = StandardScaler()
min_max = MinMaxScaler()
yeo_power_scale = PowerTransformer()
max_abs = MaxAbsScaler()
norm = Normalizer()

In [5]:
in_std_scale = pd.DataFrame(std_scale.fit_transform(inpatient_data[in_standardizing]),
                            columns=in_standardizing)
min_max_scale = pd.DataFrame(min_max.fit_transform(inpatient_data[in_standardizing]),
                             columns=in_standardizing)
yeo_in_scale = pd.DataFrame(yeo_power_scale.fit_transform(inpatient_data[in_standardizing]),
                            columns=in_standardizing)
max_abs_scale = pd.DataFrame(max_abs.fit_transform(inpatient_data[in_standardizing]),
                             columns=in_standardizing)

scale = [in_std_scale, min_max_scale, yeo_in_scale, max_abs_scale]

In [7]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=[48, 30], dpi=300)

for i in range(len(in_standardizing)):
    for j in range(len(scale)):
        sns.kdeplot(x=scale[j][in_standardizing[i]],
                    ax=axes[i][j], color='#0D0D0D', shade=True)

plt.savefig('../../demo/Fraud Detection/scaling_methods_comp.png')
plt.show()

In [6]:
encode = OrdinalEncoder()
encode.fit_transform(np.asarray(inpatient_data['Provider']).reshape(-1, 1))

array([[1543.],
       [1540.],
       [1585.],
       ...,
       [1743.],
       [1735.],
       [1177.]])

In [7]:
label = pd.read_csv('../../data/Train-1542865627584.csv', index_col='Provider')
label

,PotentialFraud
Provider,
PRV51001,No
PRV51003,Yes
PRV51004,No
PRV51005,Yes
PRV51007,No
...,...
PRV57759,No
PRV57760,No
PRV57761,No


In [8]:
inpatient_data.set_index('Provider', inplace=True)
outpatient_data.set_index('Provider', inplace=True)

In [9]:
inpatient_data = inpatient_data.join(label, on='Provider', how='left')
outpatient_data = outpatient_data.join(label, on='Provider', how='left')

In [10]:
inpatient_data.reset_index(inplace=True)
outpatient_data.reset_index(inplace=True)

In [11]:
inpatient_data.loc[:, in_standardizing] = std_scale.fit_transform(inpatient_data[in_standardizing])
inpatient_data.loc[:, ['Provider', 'PotentialFraud']] = encode.fit_transform(
    inpatient_data[['Provider', 'PotentialFraud']])
features_to_std = ['Provider', 'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician', 'OtherPhysician',
                   'ClmAdmitDiagnosisCode', 'DeductibleAmtPaid', 'DiagnosisGroupCode', 'ClmDiagnosisCode_1',
                   'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
                   'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
                   'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3',
                   'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6', 'Gender', 'Race',
                   'RenalDiseaseIndicator', 'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
                   'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
                   'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', 'ChronicCond_Diabetes',
                   'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
                   'ChronicCond_stroke']
inpatient_data.loc[:, features_to_std] = std_scale.fit_transform(inpatient_data[features_to_std])
inpatient_data

,Provider,ClaimID,ClaimStartDt,ClaimEndDt,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,0.916366,CLM46614,2009-04-12,2009-04-18,1.544420,0.781227,0.465093,0.361812,2009-04-12,-0.650436,...,-0.496801,-0.408989,0.736315,-1.332307,-2.210878,1.051783,0.780482,-0.465198,-0.484453,1.0
1,0.911242,CLM66048,2009-08-31,2009-09-02,-0.493827,0.781227,0.464462,0.361812,2009-08-31,2.217141,...,-0.496801,-0.408989,0.736315,-1.332307,-2.210878,1.051783,0.780482,-0.465198,-0.484453,0.0
2,0.988108,CLM68358,2009-09-17,2009-09-20,-0.493827,0.781227,0.465093,-2.735180,2009-09-17,-0.649996,...,-0.496801,-0.408989,0.736315,-1.332307,-2.210878,1.051783,0.780482,-0.465198,-0.484453,0.0
3,-0.796883,CLM38412,2009-02-14,2009-02-22,-0.493827,0.781227,0.464462,-2.735180,2009-02-14,-0.641644,...,-0.496801,2.445051,0.736315,-1.332307,-2.210878,-0.713398,-0.485800,-0.417088,-0.220773,0.0
4,1.275073,CLM63689,2009-08-13,2009-08-30,-0.008530,0.781227,0.464462,0.361812,2009-08-13,2.217141,...,2.012877,-0.408989,0.736315,0.750578,0.452309,0.212468,0.147341,-0.450005,-0.452812,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40469,-0.204164,CLM69886,2009-09-28,2009-10-02,-0.299708,-1.285358,-2.157821,0.361812,2009-09-28,-0.560755,...,2.012877,-0.408989,0.736315,-1.332307,0.452309,-0.348973,0.147341,-0.348720,-0.262962,1.0
40470,0.438091,CLM74504,2009-11-03,2009-11-06,-0.590886,-1.281242,-2.157347,0.361812,2009-11-03,-0.635050,...,2.012877,-0.408989,0.736315,0.750578,0.452309,-0.770339,-0.485800,-0.480391,-0.558284,0.0
40471,1.257991,CLM76485,2009-11-18,2009-11-22,-0.687946,0.781227,0.464462,0.361812,2009-11-18,0.059534,...,-0.496801,-0.408989,-1.358114,-1.332307,0.452309,-0.542574,0.147341,0.005778,-0.136396,1.0
40472,1.244326,CLM79949,2009-12-17,2009-12-18,-0.493827,-1.278607,0.465093,0.361812,2009-12-17,-0.226652,...,-0.496801,-0.408989,-1.358114,-1.332307,0.452309,-0.542574,0.147341,0.005778,-0.136396,1.0


In [14]:
outpatient_data.dtypes

Provider                                   object
ClaimID                                    object
ClaimStartDt                       datetime64[ns]
ClaimEndDt                         datetime64[ns]
InscClaimAmtReimbursed                      int64
AttendingPhysician                        float64
OperatingPhysician                        float64
OtherPhysician                            float64
ClmDiagnosisCode_1                        float64
ClmDiagnosisCode_2                        float64
ClmDiagnosisCode_3                        float64
ClmDiagnosisCode_4                        float64
ClmDiagnosisCode_5                        float64
ClmDiagnosisCode_6                        float64
ClmDiagnosisCode_7                        float64
ClmDiagnosisCode_8                        float64
ClmDiagnosisCode_9                        float64
ClmDiagnosisCode_10                       float64
ClmProcedureCode_1                        float64
ClmProcedureCode_2                        float64


In [12]:
out_encode = OrdinalEncoder()
outpatient_data.loc[:, ['Provider', 'PotentialFraud']] = encode.fit_transform(
    outpatient_data[['Provider', 'PotentialFraud']])

In [13]:
features_to_std = ['Provider', 'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
                   'OtherPhysician', 'ClmAdmitDiagnosisCode', 'DeductibleAmtPaid',
                   'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4',
                   'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
                   'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
                   'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6',
                   'Gender', 'Race', 'RenalDiseaseIndicator', 'State', 'County', 'NoOfMonths_PartACov',
                   'NoOfMonths_PartBCov', 'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
                   'ChronicCond_KidneyDisease', 'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
                   'ChronicCond_Depression', 'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
                   'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke',
                   'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
                   'OPAnnualDeductibleAmt']
outpatient_data.loc[:, features_to_std] = std_scale.fit_transform(outpatient_data[features_to_std])
outpatient_data

,Provider,ClaimID,ClaimStartDt,ClaimEndDt,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,0.919448,CLM624349,2009-10-11,2009-10-11,-0.369341,0.411377,0.460453,0.721309,0.954373,-1.063131,...,1.519701,1.739696,0.678119,0.666093,0.326615,-0.401503,-0.438037,-0.587883,-0.605775,1.0
1,1.736769,CLM189947,2009-02-12,2009-02-12,-0.297298,0.411377,0.460453,0.721309,0.954373,1.192917,...,1.519701,-0.574813,0.678119,0.666093,0.326615,-0.401503,-0.438037,-0.572399,-0.615720,0.0
2,1.729072,CLM438021,2009-06-27,2009-06-27,-0.398158,0.411377,0.460453,0.721309,-1.044738,1.192917,...,1.519701,-0.574813,0.678119,0.666093,0.326615,-0.401503,-0.438037,-0.572399,-0.615720,0.0
3,0.919448,CLM121801,2009-01-06,2009-01-06,-0.354932,0.411377,0.460453,0.721309,0.954373,1.192917,...,-0.658024,-0.574813,-1.474668,-1.501291,0.326615,-0.401503,-0.438037,-0.128521,0.100338,1.0
4,0.919448,CLM150998,2009-01-22,2009-01-22,-0.124396,0.411377,0.460453,0.721309,0.954373,-1.061435,...,-0.658024,-0.574813,-1.474668,-1.501291,0.326615,-0.401503,-0.438037,-0.128521,0.100338,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517732,-0.270146,CLM510792,2009-08-06,2009-08-06,0.740116,0.411377,-2.164812,-0.994085,0.954373,0.007703,...,-0.658024,1.739696,0.678119,-1.501291,0.326615,-0.401503,-0.438037,0.816008,1.204261,0.0
517733,-0.268047,CLM551294,2009-08-29,2009-08-29,0.163775,0.411377,-2.164812,0.721309,0.954373,0.007703,...,-0.658024,1.739696,0.678119,-1.501291,0.326615,-0.401503,-0.438037,0.816008,1.204261,0.0
517734,-0.284141,CLM596444,2009-09-24,2009-09-24,-0.326115,-2.433213,0.460453,0.721309,-1.063439,-1.036714,...,-0.658024,1.739696,0.678119,-1.501291,0.326615,-0.401503,-0.438037,0.816008,1.204261,0.0
517735,-0.275744,CLM636992,2009-10-18,2009-10-18,-0.311706,0.411377,0.460453,-0.994085,-0.989084,1.192917,...,-0.658024,1.739696,0.678119,-1.501291,0.326615,-0.401503,-0.438037,0.816008,1.204261,0.0


In [14]:
inpatient_data.drop(labels=['DOD', 'ClaimID'], axis=1, inplace=True)
outpatient_data.drop(labels=['DOD', 'ClaimID'], axis=1, inplace=True)

In [15]:
inpatient_data['ClaimDuration'] = (inpatient_data['ClaimEndDt'] - inpatient_data['ClaimStartDt']).dt.days
inpatient_data['StayDuration'] = (inpatient_data['DischargeDt'] - inpatient_data['AdmissionDt']).dt.days
outpatient_data['ClaimDuration'] = (outpatient_data['ClaimEndDt'] - outpatient_data['ClaimStartDt']).dt.days


In [16]:
inpatient_data.shape, outpatient_data.shape

((40474, 54), (517737, 50))

****Feature Selection****

In [17]:
in_Y = inpatient_data['PotentialFraud']
in_X = inpatient_data.drop(['PotentialFraud', 'ClaimStartDt', 'ClaimEndDt',
                            'AdmissionDt', 'DischargeDt', 'DOB'], axis=1)

In [ ]:
rfc = RandomForestClassifier()
rfecv = RFECV(estimator=rfc, cv=StratifiedKFold(n_splits=3), scoring='accuracy')
rfecv.fit(in_X, in_Y)

In [ ]:
plt.figure(figsize=[12, 6], dpi=300)
sns.lineplot(x=range(1, len(rfecv.grid_scores_) + 1), y=rfecv.grid_scores_)
plt.xlabel('feature count')
plt.ylabel('score')
plt.savefig('../../demo/Fraud Detection/rfecv_feture_selection.png')
plt.show()

In [21]:
kbest_mic = SelectKBest(mutual_info_classif, k=10).fit(in_X, in_Y)
kbest_f = SelectKBest(f_classif, k=10).fit(in_X, in_Y)

c:\users\ashen\appdata\local\programs\python\python37\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [ 6 23] are constant.
  UserWarning)
c:\users\ashen\appdata\local\programs\python\python37\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
plt.figure(figsize=[12, 6], dpi=300)
sns.barplot(x=list(range(1, len(kbest_mic.scores_) + 1)), y=-np.log10(kbest_mic.scores_), color='#0D0D0D')
sns.barplot(x=list(range(1, len(kbest_f.scores_) + 1)), y=-np.log10(kbest_f.scores_), color='#A69897')
plt.xlabel('feature')
plt.ylabel('score')
plt.savefig('../../demo/Fraud Detection/kbest_feture_selection.png')
plt.show()

In [36]:
estimator = ExtraTreesClassifier()
estimator.fit(in_X,in_Y)
estimator.feature_importances_

array([3.85237412e-02, 2.62571746e-02, 1.49073966e-01, 5.35243209e-02,
       1.10482853e-02, 2.53433087e-02, 0.00000000e+00, 2.59996320e-02,
       2.49279737e-02, 2.52842788e-02, 2.50590875e-02, 2.52115543e-02,
       2.48781598e-02, 2.46082025e-02, 2.37907285e-02, 2.31210592e-02,
       2.20518317e-02, 9.21258478e-03, 1.93725422e-02, 8.15145162e-03,
       2.52001221e-03, 4.23495475e-04, 7.47731958e-05, 0.00000000e+00,
       1.50474139e-02, 1.39712350e-02, 1.29149727e-02, 3.44748139e-02,
       3.42689007e-02, 1.62529058e-03, 2.22365329e-03, 1.47912221e-02,
       1.14958076e-02, 1.37483136e-02, 1.24301278e-02, 1.51274409e-02,
       1.63932855e-02, 1.15606646e-02, 1.01420717e-02, 1.41724147e-02,
       1.46754962e-02, 1.16025803e-02, 2.65984019e-02, 2.11193234e-02,
       2.66457913e-02, 2.64533340e-02, 2.50079545e-02, 2.50513258e-02])

In [ ]:
plt.figure(figsize=[12, 6], dpi=300)
sns.barplot(x=list(range(1, len(estimator.feature_importances_) + 1)),
            y=estimator.feature_importances_,
            color='#0D0D0D')
plt.xlabel('feature')
plt.ylabel('score')
plt.savefig('../../demo/Fraud Detection/frommodel_feture_selection.png')
plt.show()